### Project 1

#### Part A

Jerônimo de Abreu Afrange

- Select 30 stocks in the S&P 500 universe
- Collect daily returns for the past 10 years (2014-2023)
- Create both value-weighted and equally weighted portfolio
- Rebalance these portfolios in the first day of the month based on the information on the last day (1-day lag)
- Which portfolio has the highest turnover?
- Compute daily returns for both portfolios
- Compute the following statistics for these portfolios:
    - Annualized average return
    - Annualized standar deviation
    - Sharpe ratio
    - Information ratio (vs S&P500)
- Plot cumulative returns for both portfolios, S&P500 and risk-free




In [57]:
import pandas as pd
import dateutil.relativedelta

In [58]:
# AUXILIARY FUNCTIONS

# calculates the month number
def month_number(date, start_date):
    delta = dateutil.relativedelta.relativedelta(date, start_date)
    return delta.years * 12 + delta.months

In [59]:
# DATA LOADING AND INITIAL MANIPULATION

# creates the DataFrames for de S&P500 index and de stock prices
stocks_data = pd.read_csv("../data/stock_prices.csv")
index_data = pd.read_csv("../data/sp500_index.csv")

# removes the stocks of which we don't have data from 2014 to the current day
incomplete_stocks = stocks_data["Ticker"].value_counts()
incomplete_stocks = incomplete_stocks[incomplete_stocks < incomplete_stocks.max()]
stocks_data = stocks_data[~stocks_data["Ticker"].isin(incomplete_stocks.index)]

In [60]:
# CALCULATION OF THE DAILY RETURNS AND CREATION OF THE MONTH NUMBER COLUMN

# creation of the daily return column
stocks_data["RETURN"] = stocks_data["PX_LAST"] / stocks_data["PX_OPEN"] - 1

# creation of the month number column
stocks_data["Date"] = pd.to_datetime(stocks_data["Date"])
stocks_data["MONTH_NUMBER"] = stocks_data["Date"].apply(lambda x: month_number(x, stocks_data["Date"].min()))

# defines a few constants
INITIAL_DATE = stocks_data["Date"].min()
INITIAL_PORT_VALUE = 1e6

In [61]:
# CREATION OF THE VALUE WEIGHTED PORTFOLIO

# creates the porfolio DataFrame, which is a copy of the current stocks_data DataFrame
portfolio_vw = stocks_data.copy()

# creation of the column for the total market cap of the stocks and column of the share of the total
portfolio_vw["TOTAL_MKT_CAP"] = portfolio_vw.groupby("Date")["CUR_MKT_CAP"].transform("sum")
portfolio_vw["MKT_CAP_SHARE"] = portfolio_vw["CUR_MKT_CAP"] / portfolio_vw["TOTAL_MKT_CAP"]

# creation of the initial porfolio stock ammount for each stock, value for each stock position and total portfolio value
initial_date = portfolio_vw["Date"] == INITIAL_DATE
portfolio_vw["PORT_STOCK_AMT"] = None
portfolio_vw["PORT_STOCK_MKT_VAL_LAST"] = None
portfolio_vw.loc[initial_date, "PORT_STOCK_AMT"] = (INITIAL_PORT_VALUE * portfolio_vw["MKT_CAP_SHARE"]) / portfolio_vw["PX_OPEN"]

# propagate the stock positions to the whole month period
initial_values = portfolio_vw[initial_date][["Ticker", "PORT_STOCK_AMT"]]
portfolio_vw = portfolio_vw.drop(columns="PORT_STOCK_AMT")
merged_values = portfolio_vw[portfolio_vw["MONTH_NUMBER"] == 0]
portfolio_vw = portfolio_vw[portfolio_vw["MONTH_NUMBER"] != 0]
merged_values = merged_values.merge(initial_values, on="Ticker")
portfolio_vw = pd.concat([portfolio_vw, merged_values])
portfolio_vw["PORT_STOCK_MKT_VAL_LAST"] = portfolio_vw["PORT_STOCK_AMT"] * portfolio_vw["PX_LAST"]
portfolio_vw["PORT_TOTAL_MKT_VAL_LAST"] = portfolio_vw.groupby("Date")["PORT_STOCK_MKT_VAL_LAST"].transform("sum")


In [64]:
portfolio_vw

,Date,PX_LAST,PX_OPEN,PX_HIGH,PX_LOW,PX_VOLUME,CUR_MKT_CAP,Ticker,RETURN,MONTH_NUMBER,TOTAL_MKT_CAP,MKT_CAP_SHARE,PORT_STOCK_MKT_VAL_LAST,PORT_STOCK_AMT,PORT_TOTAL_MKT_VAL_LAST
21,2014-02-03,69.990,72.660,72.730,69.850,11966489.0,122472.5124,DIS,-0.036746,1,9.822805e+05,0.124682,NaN,NaN,0
22,2014-02-04,71.050,70.940,71.250,69.880,11829723.0,124327.3612,DIS,0.001551,1,9.879017e+05,0.125850,NaN,NaN,0
23,2014-02-05,71.760,72.050,72.050,70.370,11631868.0,125727.7597,DIS,-0.004025,1,9.885362e+05,0.127186,NaN,NaN,0
24,2014-02-06,75.560,75.610,76.730,74.780,19822855.0,132385.5842,DIS,-0.000661,1,1.014054e+06,0.130551,NaN,NaN,0
25,2014-02-07,75.670,75.080,75.700,74.610,11526738.0,132578.3107,DIS,0.007858,1,1.026975e+06,0.129096,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,2014-01-27,9.786,9.847,9.940,9.654,28162750.0,15141.4047,CMG,-0.006195,0,9.828296e+05,0.015406,14873.777628,1519.903702,975501.731946
563,2014-01-28,9.837,9.839,9.976,9.775,19901350.0,15220.6224,CMG,-0.000203,0,9.953198e+05,0.015292,14951.292717,1519.903702,987895.150641
564,2014-01-29,9.690,9.800,9.841,9.617,32888450.0,14992.5622,CMG,-0.011224,0,9.829104e+05,0.015253,14727.866873,1519.903702,975557.610262
565,2014-01-30,9.879,9.739,9.989,9.708,64118100.0,15285.2962,CMG,0.014375,0,1.002200e+06,0.015252,15015.128673,1519.903702,994717.831211
